## **執行YOLOv7 (Run YOLOv7 on CPU/GPU)**

YOLOV7 是物件偵測應用設計的 YOLO 算法的最新流行版本，以驚人的FPS實現了高mAP。

### **教程 (Tutorial)**

我們將實施以下步驟：

1. 安裝 YOLOv7 依賴項。
2. 對單個**影像**進行推理與預測。
3. 下載示例**影片**並在其上運行 YOLOv7 推理與預測。
4. 在**網絡攝影機**上運行 YOLOv7。

### 01. 安裝YOLOv7依賴項 (Setting up Dependencies)

In [ ]:
# 如果使用Google Colab
%%bash
git clone https://github.com/WongKinYiu/yolov7.git
cd yolov7
wget https://raw.githubusercontent.com/WongKinYiu/yolov7/u5/requirements.txt
pip install -r requirements.txt

In [ ]:
# 如果使用本機安裝，請用命令提示字元執行(移除驚嘆號)
! git clone https://github.com/WongKinYiu/yolov7.git
! cd yolov7
! pip install -r requirements.txt

In [ ]:
# 在yolov7資料夾中建立weights資料夾
import os
if not os.path.isdir("weights"):
    os.makedirs("weights")

* 下載yolov7.pt的權重[[連結]](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt)，並儲存至yolov7/weights資料夾中

In [1]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

C:\Users\j8288\.conda\envs\yolov7\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # 滿足步幅倍數約束，並調整圖像大小和填充圖像
    # Resize and pad image while meeting stride-multiple constraints
    
    shape = img.shape[:2]  
    
    # 判斷變數是否是一個已知的型別
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # 比率(新/原來) 
    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    
    # 只縮小，不放大（為了更好的測試結果 mAP）
    # only scale down, do not scale up (for better test mAP)
    if not scaleup:  
        r = min(r, 1.0)

    # 計算填充 (Compute padding)
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [63]:
# 您可以提供按名稱過濾的類別列表，並不用輸入類別編號，例如 ['train','person']
# You can give list of classes to filter by name, Be happy you don't have to put class number. ['train','person' ]

#classes_to_filter = ['train'] 
classes_to_filter = []

# opt字典: 儲存參數
opt  = {
    
    # 權重模型位置 (Path to weights file default weights are for nano model)
    #"weights": "weights/yolov7.pt", 
    "weights": "weights/best.pt", 
    
    # 標籤名稱
    #"yaml"   : "data/coco.yaml",
    "yaml"   : "data/data.yaml",
    
    # 輸入影像大小 (default image size)
    "img-size": 640, 
    
    # 預測的置信度臨界值 (confidence threshold for inference)
    "conf-thres": 0.25, 
    
    # 非最大抑制的IoU臨界值 (NMS IoU threshold for inference)
    "iou-thres" : 0.45,
    
    # 設備是否(不)啟用GPU(CPU)
    "device" : 'cpu',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    
    # 不顯示的類別 (list of classes to filter or None)
    "classes" : classes_to_filter
}

In [68]:
# 給與影像來源位置 (Give path of source image)
#source_image_path = 'inference/images/horses.jpg'
source_image_path = 'inference/images/1.jpg'

with torch.no_grad():
    
    weights, imgsz = opt['weights'], opt['img-size']
    #set_logging()
    device = select_device(opt['device'])
    half = device.type != 'cpu'
    print("device.type != cpu:", half)
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    print("stride:", stride)
    imgsz = check_img_size(imgsz, s=stride)  # check img_size
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

    img0 = cv2.imread(source_image_path)
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # 推斷 (Inference)
    t1 = time_synchronized()
    pred = model(img, augment= False)[0]
    
    # Apply NMS
    classes = None
    if opt['classes']:
        classes = []
        for class_name in opt['classes']:
            classes.append(names.index(class_name))

    if classes:
        classes = [i for i in range(len(names)) if i not in classes]
    
    #pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= [17], agnostic= False)
    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], agnostic= False)
    t2 = time_synchronized()
    for i, det in enumerate(pred):
        s = ''
        s += '%gx%g ' % img.shape[2:]  
        gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
    
            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

device.type != cpu: False
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
stride: 32


In [69]:
# 顯示圖片
cv2.imshow('My Image', img0)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

* numpy.ascontiguousarray(): 將一個內存不連續存儲的數組轉換為內存連續存儲的數組，使得運行速度更快
* torch.from_numpy(array): 將numpy的陣列轉為torch中的張量
* torch.Tensor.ndimension(): 計算張量的維度
* torch.Tensor.unsqueeze(i): 在張量中第i個增加一個維度

In [66]:
with torch.no_grad():
    weights, imgsz = opt['weights'], opt['img-size']
    device = select_device(opt['device'])
    half = device.type != 'cpu'
    model = attempt_load(weights, map_location=device)  
    stride = int(model.stride.max())  
    imgsz = check_img_size(imgsz, s=stride)  
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))
    
    classes = None
    if opt['classes']:
        classes = []
        for class_name in opt['classes']:
            classes.append(names.index(class_name))

    if classes:
        classes = [i for i in range(len(names)) if i not in classes]
    
def yolov7_inference(img0):
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device).float() 
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment= False)[0]
    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], agnostic= False)

    for i, det in enumerate(pred):
        s = ''
        s += '%gx%g ' % img.shape[2:]  
        gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)
    return img0

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


In [67]:
VIDEO_IN = cv2.VideoCapture(0)
while True:
    hasFrame, img = VIDEO_IN.read()
    #img = cv2.resize(img, None, fx=0.6, fy=0.6)
    img = yolov7_inference(img)
    cv2.imshow("Frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
VIDEO_IN.release()
cv2.destroyAllWindows()